## Asset Location Comparison

Investment in a dividend-paying stock and/or a taxable coupon bond is made in a brokerage account, a 401(k) account, or a Roth IRA account.
- Assumes ordinary income taxes are constant from $t=0$ to $t=T-1$ and then jump at $t=T$.
- Assumes constant dividend yield and capital gain for stocks
- Assumes constant return for bond

In [1]:
import pandas as pd
import numpy as np

Parameters

In [2]:
T_OI_0 = 0.35     # Initial tax rate on ordinary income
T_OI_T = 0.25     # Ending tax rate on ordinary income
T_DIV  = 0.15     # Tax rate on dividends
T_CG   = 0.15     # Tax rate on capital gains
DY     = 0.02     # Stock dividend yield
CG     = 0.04     # Stock capital gain
BNDRET = 0.03     # Bond rate of return
T      = 30       # Years saving

# Portfolio 1: weights of stock and bond in each asset location
WGT_BROK_STOCK = 0.25
WGT_BROK_BOND  = 0.25
WGT_401K_STOCK = 0.25
WGT_401K_BOND  = 0.25
WGT_ROTH_STOCK = 0.00
wgt_roth_bond  = 1 - WGT_BROK_STOCK - WGT_BROK_BOND - WGT_401K_STOCK - WGT_401K_BOND - WGT_ROTH_STOCK
wgts1 = [WGT_BROK_STOCK, WGT_BROK_BOND, WGT_401K_STOCK, WGT_401K_BOND, WGT_ROTH_STOCK, wgt_roth_bond]

# Portfolio 2: weights of stock and bond in each asset location
WGT_BROK_STOCK = 0.25
WGT_BROK_BOND  = 0.25
WGT_401K_STOCK = 0.25
WGT_401K_BOND  = 0.00
WGT_ROTH_STOCK = 0.25
wgt_roth_bond  = 1 - WGT_BROK_STOCK - WGT_BROK_BOND - WGT_401K_STOCK - WGT_401K_BOND - WGT_ROTH_STOCK
wgts2 = [WGT_BROK_STOCK, WGT_BROK_BOND, WGT_401K_STOCK, WGT_401K_BOND, WGT_ROTH_STOCK, wgt_roth_bond]

Future value of $1 invested in dividend-paying stock (with reinvestment)

In [3]:
rs_brok = 1 + DY*(1-T_DIV) + CG
rets_brok = (rs_brok**T)*(1-T_CG) + T_CG*(1+DY*(1-T_DIV)*(1-rs_brok**T)/(1 - rs_brok))

rs_401k = DY + CG
rets_401k = ((1-T_OI_T)*(1+rs_401k)**T) / (1 - T_OI_0)

rs_roth = DY + CG
rets_roth = (1+rs_roth)** T


print('After-tax future value of a dividend-paying stock held in:')
print(f'     Brokerage:\t{rets_brok:,.2f}')
print(f'     401k:\t{rets_401k:,.2f}')
print(f'     Roth:\t{rets_roth:,.2f}')

After-tax future value of a dividend-paying stock held in:
     Brokerage:	4.83
     401k:	6.63
     Roth:	5.74


Future value of $1 invested in taxable coupon bond (with reinvestment)

In [4]:
# Returns on Taxable coupon bond (with reinvestment)
retb_brok = (1 + BNDRET*(1-T_OI_0))**(T-1) * (1 + BNDRET*(1-T_OI_T))
retb_401k = ((1-T_OI_T)*(1 + BNDRET)**T) / (1-T_OI_0)
retb_roth = (1 + BNDRET)**T

print('After-tax future value of a taxable coupon bond held in:')
print(f'     Brokerage:\t{retb_brok:,.2f}')
print(f'     401k:\t{retb_401k:,.2f}')
print(f'     Roth:\t{retb_roth:,.2f}')

After-tax future value of a taxable coupon bond held in:
     Brokerage:	1.79
     401k:	2.80
     Roth:	2.43


Future value based on assumed portfolio allocation

In [5]:
idx  = pd.MultiIndex.from_product([['brokerage','401k','roth'],['stock','bond']])
cols = ['return', 'allocation1','fv1', 'allocation2','fv2' ]
df = pd.DataFrame(dtype=float,columns=cols,index=idx)
df['return'] = [rets_brok, retb_brok, rets_401k, retb_401k, rets_roth, retb_roth]

df['allocation1'] = wgts1
df['fv1'] = df['allocation1']*df['return']
df['allocation2'] = wgts2
df['fv2'] = df['allocation2']*df['return']
np.round(df,2)

return  allocation1   fv1  allocation2   fv2
brokerage stock    4.83         0.25  1.21         0.25  1.21
          bond     1.79         0.25  0.45         0.25  0.45
401k      stock    6.63         0.25  1.66         0.25  1.66
          bond     2.80         0.25  0.70         0.00  0.00
roth      stock    5.74         0.00  0.00         0.25  1.44
          bond     2.43         0.00  0.00         0.00  0.00

In [6]:
print('Portfolio 1:')
print(f'The ending subtotals in each account are:')
print(np.round(df.groupby(level=0)[['fv1']].sum(),2))
print(f'\nAn initial $1 turns into total FV of ${df.fv1.sum():,.2f}\n\n')

print('Portfolio 2:')
print(f'The ending subtotals in each account are:')
print(np.round(df.groupby(level=0)[['fv2']].sum(),2))
print(f'\nAn initial $1 turns into total FV of ${df.fv2.sum():,.2f}')

Portfolio 1:
The ending subtotals in each account are:
            fv1
401k       2.36
brokerage  1.65
roth       0.00

An initial $1 turns into total FV of $4.01


Portfolio 2:
The ending subtotals in each account are:
            fv2
401k       1.66
brokerage  1.65
roth       1.44

An initial $1 turns into total FV of $4.75
